### Setting Up the Database and Stage 

In [ ]:
-- Create Stage for Analyst

CREATE DATABASE CHATBOT_WORKSHOP_DB;


CREATE OR REPLACE STAGE CORTEX_ANALYST_STG ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );

create or replace TABLE CHATBOT_WORKSHOP_DB.PUBLIC.ZENITH_MERIDIAN_SUPPORT_CALLS (
	TICKET_ID NUMBER(38,0),
	TICKET_STATUS VARCHAR(16777216),
	RESOLUTION_NOTES VARCHAR(16777216),
	CHANNEL VARCHAR(16777216),
	SENTIMENT VARCHAR(16777216),
	FIRST_CONTACT_RESOLUTION BOOLEAN,
	FOLLOW_UP_REQUIRED BOOLEAN,
	ESCALATED BOOLEAN,
	PRODUCT_CATEGORY VARCHAR(16777216),
	RESPONSE_TIME_SECONDS NUMBER(38,0),
	CALL_DURATION_SECONDS NUMBER(38,0)
);


In [ ]:
-- Create table 
CREATE OR REPLACE STAGE CORTEX_ANALYST_EXT_STG
url = 's3://holworkshopbucket/cortex-analyst/'
file_format = (type = csv);

In [ ]:
USE DATABASE chatbot_workshop_db;

LIST @CORTEX_ANALYST_EXT_STG;

### Loading the Data into Snowflake Table For the Analyst to Ask Questions

In [ ]:
# Imports the Snowpark library, which provides the functionality to work with DataFrames in a Snowflake environment.
import snowflake.snowpark as snowpark 

# Imports types that are used to define the schema of the DataFrame
from snowflake.snowpark.types import StructType, StructField, IntegerType, StringType 

# Imports the function to retrieve the active Snowpark session, which is necessary for executing Snowpark operations.
from snowflake.snowpark.context import get_active_session

# This line retrieves the currently active Snowpark session, which is used to perform operations like reading data from Snowflake.
session = get_active_session()


csv_file_path = "@CORTEX_ANALYST_EXT_STG/zenith_meridian_support_calls_dataset.csv"

dataframe = session.read.options({
        "skip_header": 1
    }).csv(csv_file_path)


dataframe.write.insert_into("ZENITH_MERIDIAN_SUPPORT_CALLS")
session.table("ZENITH_MERIDIAN_SUPPORT_CALLS").collect()

In [ ]:
select * from ZENITH_MERIDIAN_SUPPORT_CALLS;

### Configuring Semantic Model - YAML File to Help the Cortex Analyst to understand our snowflake data table/tables.

In [ ]:
COPY FILES INTO @CORTEX_ANALYST_STG FROM @CORTEX_ANALYST_EXT_STG/ticket.yaml;

In [ ]:
LIST @CORTEX_ANALYST_STG;

### Questions

1.Top 2 products with high average call duration \
2.Show the top 3 Categories which has the most open tickets \
3.Which Channel has the lowest average call duration

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
WITH __zenith_meridian_support_calls AS (
  SELECT
    product_category,
    call_duration_seconds
  FROM chatbot_workshop_db.public.zenith_meridian_support_calls
)
SELECT
  product_category,
  AVG(call_duration_seconds) AS avg_call_duration
FROM __zenith_meridian_support_calls
GROUP BY
  product_category
ORDER BY
  avg_call_duration DESC NULLS LAST
LIMIT 2
 -- Generated by Cortex Analyst
;

In [ ]:
WITH __zenith_meridian_support_calls AS (
  SELECT
    sentiment,
    product_category
  FROM chatbot_workshop_db.public.zenith_meridian_support_calls
)
SELECT
  product_category,
  COUNT(*) AS negative_sentiment_count
FROM __zenith_meridian_support_calls
WHERE
  sentiment = 'Negative'
GROUP BY
  product_category
ORDER BY
  negative_sentiment_count DESC NULLS LAST
LIMIT 1
 -- Generated by Cortex Analyst
;

In [ ]:
WITH __zenith_meridian_support_calls AS (
  SELECT
    channel,
    call_duration_seconds
  FROM chatbot_workshop_db.public.zenith_meridian_support_calls
)
SELECT
  channel,
  AVG(call_duration_seconds) AS avg_call_duration
FROM __zenith_meridian_support_calls
GROUP BY
  channel
ORDER BY
  avg_call_duration ASC
LIMIT 1
 -- Generated by Cortex Analyst
;